In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time


linkedin_email = "mihran.akopyan@campus.masterschool.com"
linkedin_password = "u&ubPmd9:h%RH%x"

url = 'https://www.linkedin.com/jobs'

In [43]:
import re

def check_element (elem):
    if elem is None: 
        return ''
    if len (elem) > 0:
        return elem.text.strip()
    else:
        return ''



def extract_hourly_or_salary_range(salary_string):
    hourly_match = re.search(r'\$([\d,]+(?:\.\d+)?)\s?(?:-|to)?\s?\$?([\d,]+(?:\.\d+)?)?\s?\/\s?hr', salary_string)
    salary_match = re.search(r'\$([\d,]+(?:\.\d+)?)\s?K?\s?\/\s?yr\s?(?:-|to)?\s?\$?([\d,]+(?:\.\d+)?)?\s?K?\s?\/\s?yr', salary_string)

    if hourly_match:
        hourly_rate_lower = float(hourly_match.group(1).replace(',', ''))
        hourly_rate_upper = float(hourly_match.group(2).replace(',', '')) if hourly_match.group(2) else hourly_rate_lower

        # Assuming 40 hours per week and 52 weeks per year
        annual_lower = hourly_rate_lower * 40 * 52
        annual_upper = hourly_rate_upper * 40 * 52

        average = (annual_lower + annual_upper) / 2

        return [annual_lower, annual_upper, average]
    elif salary_match:
        salary_lower = float(salary_match.group(1).replace(',', '')) * 1000
        salary_upper = float(salary_match.group(2).replace(',', '')) * 1000 if salary_match.group(2) else salary_lower

        average = (salary_lower + salary_upper) / 2

        return [salary_lower, salary_upper, average]
    else:
        return [0,0,0]

# # Example usage
# strings = [
#     '$57/hr - $78/hr · Vision, +1 benefit',
#     '401(k), +6 benefits',
#     '$117K/yr - $134.2K/yr · Medical benefit',
#     '$55.1K/yr - $68.9K/yr',
#     '$70/hr - $75/hr · 401(k), +1 benefit',
#     'Starting at $40/hr',
#     '$57/hr - $78/hr · Vision, +1 benefit'
# ]

# for salary_string in strings:
#     result = extract_hourly_or_salary_range(salary_string)
#     if result:
#         print(f"Input: {salary_string}\nConverted Range: {result}\n")
#     else:
#         print(f"Input: {salary_string}\nUnable to extract hourly rate or salary range.\n")


In [21]:
browser = webdriver.Firefox()
browser.get(url)

email_input = browser.find_element("name", "session_key")
password_input = browser.find_element("name", "session_password")
login_button = browser.find_element("xpath", "//button[@type='submit']")

In [22]:
email_input.send_keys(linkedin_email)
password_input.send_keys(linkedin_password)

In [23]:
login_button.click()

time.sleep(5)

In [24]:
search_box = browser.find_element(By.CLASS_NAME, 'jobs-search-box__text-input')
search_box.send_keys('Data Analyst')
search_box.send_keys(Keys.RETURN)


In [28]:
date_filter = browser.find_element("id", "searchFilter_timePostedRange")
date_filter.click()


In [29]:


class_name = "t-14 t-black--light t-normal"
span_text = "week"

past_week_x_path= f"//span[@class='{class_name}' and contains(.,'{span_text}')]"
past_week = browser.find_element("xpath", past_week_x_path)
past_week.click()


In [30]:

class_name = "artdeco-button__text"
span_text = "results"
show_results_xpath = f"//span[@class='{class_name}' and contains(.,'{span_text}')]"
show_results = browser.find_element("xpath", show_results_xpath)
show_results.click()



In [48]:
jobs_block = browser.find_element(By.CSS_SELECTOR, '.scaffold-layout__list-container')
jobs_list= jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')


In [49]:
# browser.execute_script("arguments[0].scrollIntoView();", target_li_elements[24])
browser.execute_script("arguments[0].scrollIntoView();", jobs_list[len(jobs_list)-1])

In [50]:
soup = BeautifulSoup(browser.page_source, "html.parser")

In [51]:
target_class = 'jobs-search-results'
target_li_elements = soup.find_all('li', class_=lambda x: x and 'jobs-search-results__list-item' in x)

In [52]:
num = 24


job_title = check_element (target_li_elements[num].find('a', class_=lambda x: x and 'title' in x))
company = check_element ( target_li_elements[num].find('span', class_= 'job-card-container__primary-description') )
location = check_element (target_li_elements[num].find('div', class_= 'artdeco-entity-lockup__caption ember-view') )
salary = check_element (target_li_elements[num].find('div', class_= lambda x: x and 'mt1 t-sans t-12 t-black--light' in x) )
link = target_li_elements[num].find('a').get('href')

dic = {     'job_title': job_title ,
            'company': company,
            'location': location,
            'salary_string': salary,
            'lower_bound_extracted_salary':  (extract_hourly_or_salary_range(salary)[0]),
            'upper_extracted_salary':  (extract_hourly_or_salary_range(salary)[1]),
            'average_extracted_salary':  (extract_hourly_or_salary_range(salary)[2]),
            'link': 'www.linkedin.com'+link
       
          }

dic

{'job_title': 'Data Analyst, gt.school (Remote) - $100,000/year USD',
 'company': 'Crossover',
 'location': 'Coral Springs, FL (Remote)',
 'salary_string': '$50/hr',
 'lower_bound_extracted_salary': 104000.0,
 'upper_extracted_salary': 104000.0,
 'average_extracted_salary': 104000.0,
 'link': 'www.linkedin.com/jobs/view/3801851735/?eBP=CwEAAAGNEI9OQql6x-zTolAtTVGzz8wm6FEIKAa2igWgYIxU64cVpv_jnn9YxTdIHegPKa5lvXDMB0qzOYz57CNFQ1S5Pg93mKtggmj4--JwuKVn_K6xi_QNue3hDxYphBHeOPhYUK0lXoUPWzBbtvjmN0ObmGU0OhM3iywUBgP1SsoKOBNlaINPsQ8Uu_ScJ_1SnAp3juAbFShYAvMEs_qhkViqj-ofSWDwgqSYMHHllmntBc147FWL3NdG89JUJ1GQQCrcrquimb-PiQMWeTErr2Q_CeagYcX_eha3K9lBXIOXYbeNRNBb3lEXj9JXPK2gESlaaQ2nftd-8Hcua0jCs1eshDc4kglJMXOzOmjKSKPjWXtCElEfdEcaiZOYACqaZOnuxE_j3UXV78fM-p73mzR3hKYQ&refId=YWAmcbsHxqpbmcp9lZYkXQ%3D%3D&trackingId=nvS51JdslZuai9GT1ZGRAQ%3D%3D&trk=flagship3_search_srp_jobs'}

In [536]:
page = 2
next_page_xpath = "//button[@aria-label='Page "+str(page)+"']"
# next_page = browser.find_element("xpath", "//button[@aria-label='Page 3']")
next_page = browser.find_element("xpath", next_page_xpath)
next_page.click()



In [34]:
len (jobs_list)

25

In [502]:
n = 13
for i in jobs_list:
    print("__________")
    n+=1
    browser.execute_script("arguments[0].scrollIntoView();", jobs_list[n-1])
    print(n, " ",i.text)
    break

__________
14   Data Analyst
Flashii
Seattle, WA (Hybrid)
Promoted
Easy Apply


In [540]:
len (target_li_elements)

25

In [37]:
# counter = 0
# for e in target_li_elements:
#     print (str(counter) + '____________________________________________________________________________________')
#     print (str(counter) + '____________________________________________________________________________________')
#     print (str(counter) + '____________________________________________________________________________________')
#     print (str(counter) + '____________________________________________________________________________________')
    
#     print (e)
#     counter = counter +1
    


In [84]:
job_title


'100% Remote Data Analyst'

In [16]:
company = target_li_elements[0].find('div',class_='artdeco-entity-lockup__subtitle ember-view').text.strip()

In [17]:
company

'Hays'

In [18]:
location = target_li_elements[0].find('div',class_='artdeco-entity-lockup__caption ember-view').text.strip()

In [19]:
location

'Miami, FL (On-site)'

In [36]:
salary = target_li_elements[0].find_all('li',class_='job-card-container__metadata-item')[1].text.strip()
salary

'$70K/yr - $90K/yr · 401(k) benefit'

In [280]:
target_li_elements[5].find_all ('li','job-card-container__metadata-item')

[<li class="job-card-container__metadata-item">
 <!-- -->Olathe, KS<!-- -->
 </li>]

In [38]:
for elem in target_li_elements:
    job_title = elem.find('a',class_='job-card-list__title').text.strip()
    company = elem.find('div',class_='artdeco-entity-lockup__subtitle ember-view').text.strip()
    location = elem.find_all('li',class_='job-card-container__metadata-item')[0].text.strip()
    salary = elem.find_all('li',class_='job-card-container__metadata-item')[1].text.strip()
    
    
    dic = { 'job_title': job_title ,
            'company': company,
            'location': location,
            'salary': salary,
          }
    break
    

In [39]:
dic

{'job_title': 'Business Data Analyst',
 'company': 'Hays',
 'location': 'Miami, FL (On-site)',
 'salary': '$70K/yr - $90K/yr · 401(k) benefit'}

In [50]:
def check_element (elem):
    if elem is None: 
        return ''
    if len (elem) > 0:
        return elem.text.strip()
    else:
        return ''

In [41]:
dic

{'job_title': 'Business Data Analyst',
 'company': 'Hays',
 'location': 'Miami, FL (On-site)',
 'salary': '$70K/yr - $90K/yr · 401(k) benefit'}

In [111]:
job_title = target_li_elements[2].find('a', class_=lambda x: x and 'title' in x)
job_title.text

'\n                    Senior Data Analyst\n                '

In [124]:
company = target_li_elements[2].find('span', class_= 'job-card-container__primary-description')
company.text.strip()

'Cityblock Health'

In [136]:
location = target_li_elements[2].find('div', class_= 'artdeco-entity-lockup__caption ember-view')
location.text.strip()

'United States (Remote)'

In [142]:
salary = target_li_elements[2].find('div', class_= lambda x: x and 'mt1 t-sans t-12 t-black--light' in x)
salary.text.strip()

'$117K/yr - $134.2K/yr · Medical benefit'

In [38]:
# for e in target_li_elements:
#     print (e)
#     print ('---------------------')
#     print ('---------------------')
#     print ('---------------------')
#     print ('---------------------')


In [176]:
salary = check_element (target_li_elements[num].find('div', class_= lambda x: x and 'mt1 t-sans t-12 t-black--light' in x) )


In [298]:
len (target_li_elements)

35

In [174]:
x is None

True

In [47]:
x = extract_hourly_or_salary_range("$117K/yr - $134.2K/yr · Medical benefit")

In [48]:
print(x)

[117000.0, 134200.0, 125600.0]


In [254]:
link = target_li_elements[num].find('a')

In [256]:
link.get('href')

'/jobs/view/3799960108/?eBP=CwEAAAGNA6SGtlC0cwQwWB85lqZu9G0NhRGswr1CnsBt6fxUWwUvM4M9eUDUq6M2hJ4y_GCfIK2ncUiOAUgFUfXjQrpYStliYhPTx5HBsN0LiGe5bpFImI_uL87OeMEfbueFoE_VNyOGVjMUCEb6JANZA1vO_C-5AWM3OHsOTAqoffLi73iteGV8chTAV2nSZ75mGe6hYeJExyR0LHZ70x9yO2VV9knq_Dwg-eyrLbMCEQn5JJDCKbIl6ClPI68eIlb1KQscydipgt3f9cXAvuqa9P_fr6pNpjobvSNtdebw3Efm6DWPU9-juvnamp-a0RzeS8HXWzF5rGMXNPQjZS2GSTo9LfhYLOHS3Iopd89Ks78T_76E18QkCRMshss7W66hxGonk2ite_simgj8C_BfxxU4hftu&refId=zxUetQiXMGY9a7q5VOABbw%3D%3D&trackingId=gVj%2FaEjz%2FEGuiYoLmLX7fg%3D%3D&trk=flagship3_search_srp_jobs'

In [369]:
target_li_elements = soup.find_all(By.CLASS_NAME, 'occludable-update')

In [373]:
target_li_elements = soup.find_all('li', class_=lambda x: x and 'occludable-update' in x)

In [401]:
last_height = browser.execute_script('return document.body.scrollHeight')

In [402]:
browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')